In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xmltodict
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

<ipython-input-2-285f80bacdea>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from childes_mi.utils.paths import DATA_DIR, ensure_dir, PHONBANK_DIR

In [4]:
from childes_mi.utils.general import readfile

In [5]:
from childes_mi.datasets.phonbank import (
    get_participants,
    get_transcript_metadata,
    _check_for_expected_keys,
    expected_keys,
    process_record,
    process_transcript
)

### locate XML

In [6]:
PHONBANK_DIR

PosixPath('/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora')

In [7]:
all_XML = list((PHONBANK_DIR/'Eng-NA').glob('**/*.xml'))

In [8]:
len(all_XML), all_XML[:3]

(2579,
 [PosixPath('/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/Davis-CDI/project.xml'),
  PosixPath('/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/Davis-CDI/Kae-WS/Kae-1_04_14.xml'),
  PosixPath('/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/Davis-CDI/Kae-WS/Kae-1_07_29.xml')])

### test one transcript

In [9]:
transcript = all_XML[1]
#transcript = '/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/data/raw/PHON_Data_Dir/corpora/-NA/Davis-CDI/Ben-WS/Ben-1_11_06.xml'
print(transcript)

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/Davis-CDI/Kae-WS/Kae-1_04_14.xml


In [10]:
test = process_transcript(
    transcript,
)

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:119: UserWarning: media not in session/header
  warnings.warn("{} not in {}".format(elem, place))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:119: UserWarning: birthday not in participant
  warnings.warn("{} not in {}".format(elem, place))


In [11]:
from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects

In [12]:
nex = -1
with Parallel(n_jobs=1) as parallel:
    transcript_dfs = parallel(
        delayed(process_transcript)(
            transcript,
            participant_save_loc=DATA_DIR
            / ("processed-all/phonbank/participants/" + '_'.join(transcript.as_posix()[:-4].split('/')[-5:]) + ".pickle"),
            transcript_save_loc=DATA_DIR
            / ("processed-all/phonbank/transcripts/" + '_'.join(transcript.as_posix()[:-4].split('/')[-5:]) + ".pickle"),
        )
        for transcript in tqdm(all_XML[:nex])
    )

  0%|          | 0/2578 [00:00<?, ?it/s]

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:224: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/Davis-CDI/project.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:125: UserWarning: notes not an expected element in record
  warnings.warn("{} not an expected element in {}".format(elem, place))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:224: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/project.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LR

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/2b/e2bt23f122.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/2b/e2bt22m311.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/4a/e4at13f121.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/4a/e4at10f111.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/5b/e5bt16f122.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/5b/e5bt21m322.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/3b/e3bt24f222.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/3b/e3bt19m112.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/5a/e5at05m222.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA/PaidoEnglish/5a/e5at01f122.xml
  warnings.warn("No transcripts for {}".format(xml_loc))
/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/datasets/phonbank.py:270: UserWarning: No transcripts for /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/raw/PHON_Data_Dir/corpora/Eng-NA

In [13]:
transcript_df = pd.concat(transcript_dfs)

In [14]:
transcript_df[:3]

,transcript_id,transcript_root_name,corpus,version,date,language,media,xml_loc
0,Kae-1_04_14,corpora_Eng-NA_Davis-CDI_Kae-WS_Kae-1_04_14,Kae-WS,PB1.2,2004-04-04,eng,NaN,/mnt/cube/tsainbur/Projects/github_repos/LRSO_...
0,Kae-1_07_29,corpora_Eng-NA_Davis-CDI_Kae-WS_Kae-1_07_29,Kae-WS,PB1.2,2004-07-20,eng,NaN,/mnt/cube/tsainbur/Projects/github_repos/LRSO_...
0,Kae-1_05_02,corpora_Eng-NA_Davis-CDI_Kae-WS_Kae-1_05_02,Kae-WS,PB1.2,2004-04-23,eng,NaN,/mnt/cube/tsainbur/Projects/github_repos/LRSO_...


In [15]:
transcript_df.date.values

array(['2004-04-04', '2004-07-20', '2004-04-23', ..., '1980-12-12Z',
       '1981-01-16Z', '1980-12-05Z'], dtype=object)

In [16]:
len(transcript_df)

2407

In [17]:
transcript_df.to_pickle(DATA_DIR / 'processed/phonbank/transcript_df_all.pickle')

In [19]:
#for transcript in tqdm(all_XML[7000:]):
#    process_transcript(transcript, participant_save_loc=None, transcript_save_loc=None))